# **TEST THE MODEL 2**

## Installed Libraries

In [1]:
!pip install --quiet --upgrade pip
!pip uninstall --quiet tensorflow tensorflow-tensorboard tensorflow-estimator
!pip install --quiet gast==0.2.2
!pip install --quiet tensorflow-gpu

# Reference: https://stackoverflow.com/questions/69027356/importing-tensorflow-shows-errors

# for avoiding error: module 'tensorflow_core.keras.activations' has no attribute 'swish'
!pip3 install --quiet --upgrade tensorflow-gpu

     |████████████████████████████████| 2.1 MB 3.4 MB/s 
Proceed (Y/n)? Y
Proceed (Y/n)? Y
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.16.0 requires gast>=0.3.2, but you have gast 0.2.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.5/497.5 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 KB 24.6 MB/s eta 0:00:00


In [2]:
!pip install --quiet transformers==4.1.1
!pip install --quiet tokenizers==0.9.4
!pip install --quiet tensorflow
!pip install --quiet sentencepiece==0.1.94
!pip install --quiet tqdm==4.56.0
!pip install --quiet sense2vec==1.0.2
!pip install --quiet gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.2/895.2 KB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.5/497.5 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.5/253.5 KB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.9/211.9 KB 20.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.8 MB/s eta 0:00:00


In [4]:
# in case of error - re-run this cell

import torch
import tensorflow as tf
from transformers import T5ForConditionalGeneration,T5Tokenizer
import gradio as gr

In [5]:
# connect your personal google drive to store dataset and trained model
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
!ls '/content/gdrive/My Drive'

'Colab Notebooks'   DISSERTATION   Other   Uni


In [7]:
# need to change dir => so, "s2v_old" can be found and be used
%cd /content/gdrive/My Drive/DISSERTATION/

/content/gdrive/My Drive/DISSERTATION


In [8]:
# Load Sense2Vec Word Embedding
from sense2vec import Sense2Vec
s2v = Sense2Vec().from_disk('s2v_old')
from collections import OrderedDict

# Load WordNet Word Embedding
import re
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [9]:
# set seed to reproduce results. Feel free to change the seed though to get different results
# it's for the decoding strategies
tf.random.set_seed(0)

## Load the model

In [10]:
# file path of trained model
trained_model_path = '/content/gdrive/My Drive/DISSERTATION/MODEL 2/t5/model/'

# file path of trained tokenizer
trained_tokenizer_path = '/content/gdrive/My Drive/DISSERTATION/MODEL 2/t5/tokenizer/'

In [11]:
# load the fine-tuned model
model = T5ForConditionalGeneration.from_pretrained(trained_model_path)

# load the tokenizer
tokenizer = T5Tokenizer.from_pretrained(trained_tokenizer_path)

KeyboardInterrupt: ignored

In [ ]:
# in case cuda.is_available() is True => then, the device used is a GPU
# otherwise, it's a CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# print what type of the device is used (GPU or CPU)
print ("device ",device)

# move the model to the device used (in my case is GPU)
model = model.to(device)

### Function for encoding user's input (context and answer)

In [ ]:
def encode_input(text):
    # encode text for feeding it to T5 model
    # the keys of the encoded dictionary are [input_ids, attention_mask]
    encoding = tokenizer.encode_plus(text,max_length =512, padding=True, return_tensors="pt")

    # move the encoded dictionary to the used device (GPU)
    input_ids  = encoding["input_ids"].to(device)

    return input_ids

## Decoding Strategies for Question Generation

### Beam Search

In [ ]:
def beam_search(input_ids):
    """
    Beam Search Decoding Strategy
    """

    # beams refer to the decoding style used - there are several kinds of decoding methods for generated2text models 
    beam_outputs = model.generate(
        input_ids=input_ids,  # the token ids of the the "text" variable above
        max_length=72,  # max length of the output 

        num_beams=5,  # 3 distractors
        #no_repeat_ngram_size=3,  # no n-gram will appear three times => the ideal would be to be equal to 2 so no n-gram would appear twice but it ouputs errors
        num_return_sequences=1,  # generate one sequence of outputs/distractors
        early_stopping=True  # so that the generation is finished when all beam hypotheses reached the EOS token (</s>)
    )

    return beam_outputs

### Greedy Search

In [ ]:
def greedy_search(input_ids):
  """
  Greedy Search Decoding Strategy
  """

  greedy_output = model.generate(input_ids, max_length=50)

  return greedy_output

### Sampling

In [ ]:
def sampling_decoding(input_ids):
  """
  Sampling Decoding Strategy
  """

  # activate sampling and deactivate top_k by setting top_k sampling to 0
  sample_output = model.generate(
      input_ids, 
      do_sample=True, 
      max_length=72, 
      top_k=0
  )

  return sample_output

### Sampling with Temperature

In [ ]:
def sampling_with_temperature_decoding(input_ids):
  """
  Sampling with Temperature Decoding Strategy
  """

  # use temperature to decrease the sensitivity to low probability candidates
  sample_output = model.generate(
      input_ids, 
      do_sample=True, 
      max_length=72, 
      top_k=0, 
      temperature=0.7
  )

  return sample_output

### Top-K Sampling

In [ ]:
def topK_sampling(input_ids):
  """
  Top-K Sampling Decoding Method
  """

  # set top_k to 50
  sample_output = model.generate(
      input_ids, 
      do_sample=True, 
      max_length=72, 
      top_k=50
  )

  return sample_output

### Top-p (nucleus) Sampling

In [ ]:
def nucleus_sampling(input_ids):
  """
  Top-K Sampling Decoding Strategy
  """

  # deactivate top_k sampling and sample only from 92% most likely words
  sample_output = model.generate(
      input_ids, 
      do_sample=True, 
      max_length=72, 
      top_p=0.92, 
      top_k=0
  )

  return sample_output

## Functions for Generating Distractors

### WordNet Word Embedding

In [ ]:
def get_distractors_wordnet(word):
    
    """
    This function is called when the WordNet radiobutton is selected by the user

    Finds hyponynms of the given answer/word 
    Returns 3 distractors
    """

    distractors=[] # initalize a list for adding the distractors of the given word

    try:
      syn = wn.synsets(word,'n')[0] # get noun synonyms => thus, 'n'
    
      word = word.lower() # make the word lowercase
      orig_word = word # original word is the lowercased word

      # if the word can be split => replace the space with an underscore
      if len(word.split()) > 0:
          word = word.replace(" ", "_")

      # hypernym is the higher-level category => we are looking for hyponyms - sub-categories
      hypernym = syn.hypernyms()

      # if hypernym is 0 => the given word is in the higher-level category and not in the subcategory => return an empty list of distractors => no distractors found
      if len(hypernym) == 0:
          return distractors
      for item in hypernym[0].hyponyms():
          name = item.lemmas()[0].name()
          print ("name ",name)

          # check if the hyponym found is the same with the word given => if yes, check for other hyponyms
          if name == orig_word:
              continue

          # if no, replace the underscore to a space
          name = name.replace("_", " ")
          # join the splitted words to one string
          name = " ".join(w.capitalize() for w in name.split())

          # check that the found hyponym is not empty and the is not already in the list => append it to the list
          if name is not None and name not in distractors:
              distractors.append(name)
    except: # in case the given word has no synsets (set of synonyms) => exception
      print ("Wordnet distractors not found")


    # return the first 3 distractors
    return distractors[:3]

### Sense2Vec Word Embedding

In [12]:
def get_distractors_sense2vec(word):

  """
    This function is called when the Sense2Vec radiobutton is selected by the user

    returns 3 distractors
  """

  output = []
  word = word.lower()
  word = word.replace(" ", "_")

  sense = s2v.get_best_sense(word)
      
  if not sense: # check if the word has no sense => return
      return None
  else: 
      most_similar = s2v.most_similar(sense, n=3)
      

  for each_word in most_similar:
      append_word = each_word[0].split("|")[0].replace("_", " ").lower()
      
      
      if append_word.lower() != word:
          output.append(append_word.title())
      
  out = list(OrderedDict.fromkeys(output))

  return out

## Prepare GUI

In [ ]:
context = gr.inputs.Textbox(lines=2, placeholder="Enter context/passage here...")
answer = gr.inputs.Textbox(lines=1, placeholder="Enter right answer here...")
decoding_strategy = gr.inputs.Radio(["Beam Search", "Greedy Search", "Sampling", "Sampling w Temp", "Top-K", "Top-p"])
word_embedding = gr.inputs.Radio(["Wordnet", "Sense2Vec"])


displayed_output = gr.outputs.HTML(label="PASSAGE, ANSWER, QUESTION")

In [ ]:
def second_model(context, answer, decoding_strategy, word_embedding):
    text = "context: "+context + " " + "answer: " + answer + " </s>"

    input_ids = encode_input(text)

    # place the model in evaluation mode => inference
    model.eval()


    # get the generated output => the question
    if decoding_strategy == "Beam Search":
      generated_output = beam_search(input_ids)
    elif decoding_strategy == "Greedy Search":
      generated_output = greedy_search(input_ids)
    elif decoding_strategy == "Sampling":
      generated_output = sampling_decoding(input_ids)
    elif decoding_strategy == "Sampling w Temp":
      generated_output = sampling_decoding(input_ids)    
    elif decoding_strategy == "Top-K":
      generated_output = topK_sampling(input_ids)   
    elif decoding_strategy == "Top-p":
      generated_output = nucleus_sampling(input_ids)   

    # decode the generated output
    decoded_output = tokenizer.decode(generated_output[0], skip_special_tokens=True)


    # if Wordnet is selected => call get_distractors_wordnet() function => get distractors from wordnet for the specific keyword/answer
    if word_embedding=="Wordnet":
      distractors = get_distractors_wordnet(answer)
    else:
      # Othwerise, sense2vec is selected => call get_distractors() function => get distractors from Sense2Vec for the specific keyword/answer
      distractors = get_distractors_sense2vec(answer)


    # question
    # [10:] => the string part of ---"question:"---  from "question: {question}" will be skipped
    displayed_output = f"<p> <b style='color:blue;'> {decoded_output[10:]} </b> </p>" 


    # question + \n + answer
    displayed_output = f"{displayed_output}<p> <b style='color:green;'> {answer} </b> </p>"


    # question + \n + answer + \n + distractors
    if distractors is not None and len(distractors) == 3:
      for distractor in distractors:
        displayed_output = displayed_output + "<b style='color:brown;'>" + distractor+  "</b>"+"<br>"

    # question + \n + answer + \n + distractors + \n + context
    displayed_output = f"{displayed_output}<p> {context} </b> </p>"


    return displayed_output

In [ ]:
face = gr.Interface(
    fn = second_model, 
    inputs = [context, answer, decoding_strategy, word_embedding], 
    outputs = displayed_output
)

## Test GUI

In [ ]:
# if degug=True => prints the errors in the cell output
face.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://50737.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:184: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1065: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


Keyboard interruption in main thread... closing server.


(<fastapi.applications.FastAPI at 0x7f400a4f8050>,
 'http://127.0.0.1:7860/',
 'https://50737.gradio.app')